## ToG with link predictor and tail selector

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
import torch
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from utils.graph import KGraphPreproc
from utils.graph.tog_lp import ToGLPChain
from utils.llm.mistral import MistralLLM
from utils.llm.qwen import Qwen2_5
from utils.file import export_results_to_file
import os
os.environ["HF_HOME"] = "/models"
os.environ["TRANSFORMERS_CACHE"] = "/models"
device = torch.device("cuda:1")
torch.cuda.set_device(device)
torch.set_default_device(device)

### ToG Algorithm pseudocode
given a list of entities $E^{D-1}$:
1. Select all relations $R^D$
2. Select top-N relevant $E^{D-1}$-$R^D$-? | ?-$R^D$-$E^{D-1}$
3. Extend P by top-N paths
4. Decide if enough to answer the question (YES/NO)
5. Repeat (NO), return (YES)


### 0. Init

### 2. Select top relevant candidate triplets 

## QA Tests

In [ ]:
qwen = Qwen2_5()

In [7]:
mistral = MistralLLM()

In [20]:
sbert = SentenceTransformer("all-mpnet-base-v2")
sbert_cache_path = "/datasets/MetaQA/cache/sbert.csv"

In [10]:
metaqa_graph = KGraphPreproc.get_metaqa_graph()

metaqa_graph.embed_triplets(
    embedding_function=lambda q: sbert.encode(q),
    cache_path=sbert_cache_path
)

Checking embedding cache
Loading embedding cache


124680it [04:27, 465.26it/s]


In [11]:
hop = "3hop"
metaqa = pd.read_csv(f"/datasets/MetaQA/{hop}/test_1000.txt", header=None, index_col=0)
metaqa.rename(columns={1: "Question", 2: "Answers"}, inplace=True)

In [14]:
import re
regex_mqa_topic_entity = re.compile("\[(.*?)\]")
def extract_mqa_topic_entity(question):
    return regex_mqa_topic_entity.findall(question)[0]

In [199]:
chain = ToGLPChain.from_llm(
    llm=mistral,
    link_predictor_llm=qwen,
    graph=metaqa_graph,
    sbert=sbert,
    verbose=False
)

/model/utils/graph/tog_lp.py:236: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=qa_prompt)


In [182]:
experiment_name = f"tog-pl-1"
res_path = f"/datasets/MetaQA/results/{hop}/{experiment_name}.csv"

In [ ]:
results = []
id_list = []
l = 0
if os.path.isfile(res_path):
    r_df = pd.read_csv(res_path)
    l = len(r_df)
    results = list(r_df.Model.values)
for c, (i, r) in enumerate(tqdm(list(metaqa.iterrows()))):
    id_list.append(i)
    if c < l:
            continue
    q = r.Question
    topic_entity = extract_mqa_topic_entity(q)
    response = chain.invoke(
        input={
            "query": q,
            "topic_entities": [topic_entity]
    })
    answer = response["result"]["text"]
    depth = response["depth"]
    results.append((answer, depth))
    if c % 10 == 0:
        export_results_to_file(res_path, results, id_list)
export_results_to_file(res_path, results, id_list)


 90%|█████████ | 903/1000 [1:04:48<2:35:58, 96.48s/it]